### Import packages

In [1]:
# Import necessary packages
import numpy as np
import rasterio
import pywt
from skimage.transform import resize

### Load EMIT PCA and Sentinel-2 

In [2]:
# Load the images (10m Sentinel-2 and 60m original image)
s2_10m_path = r'C:\Users\User\Documents\DataFusion\DataFusion\S2_AddedIndices\uMzimvubuCatchment_datafusion.tif'
emit_pca_60m_path = r'C:\Users\User\Documents\DataFusion\DataFusion\30092014_PCA\10012024PCA_image.tif'

with rasterio.open(s2_10m_path) as src_10m:
    bands_10m = src_10m.read()  # Read Sentinel-2 bands
    meta_10m = src_10m.meta.copy()  # Copy metadata for future use
    print(f"Number of bands: {bands_10m.shape[0]}")

with rasterio.open(emit_pca_60m_path) as src_60m:
    bands_60m = src_60m.read()  # Read original 60m bands
    meta_60m = src_60m.meta.copy()  # Metadata from the 60m image
    print(f"Number of bands: {bands_60m.shape[0]}")


Number of bands: 16
Number of bands: 12


### Data fusion
#### This code cell fuses Sentinel-2 and 12 PCA-reduced EMIT banfs and resizes the 60m EMIT to 10m

In [3]:
# Create a new array to hold the fused bands (16 from Sentinel-2 + 12 from EMIT PCA)
num_bands_s2 = bands_10m.shape[0]  # 16 bands and indices from Sentinel-2
num_bands_emit = bands_60m.shape[0]  # 12 bands from EMIT PCA
fused_bands = np.zeros((num_bands_s2 + num_bands_emit, bands_10m.shape[1], bands_10m.shape[2]), dtype=np.float32)

# Copy Sentinel-2 bands to the fused array
fused_bands[:num_bands_s2, :, :] = bands_10m  # First 16 bands are from Sentinel-2

# Loop through remaining EMIT bands and resize to match the 10m resolution
for i in range(num_bands_emit):
    # Resize each EMIT band to 10m resolution
    resized_emit_band = resize(bands_60m[i], (bands_10m.shape[1], bands_10m.shape[2]), order=1, mode='reflect', anti_aliasing=True)
    # Add the resized EMIT band to the fused bands
    fused_bands[num_bands_s2 + i, :, :] = resized_emit_band


### Saves the fused image

In [4]:
# Update metadata to reflect the new fused bands
meta_fused = {
    'driver': 'GTiff',
    'count': fused_bands.shape[0],  # Total of 239 bands
    'dtype': 'float32',
    'height': fused_bands.shape[1],
    'width': fused_bands.shape[2],
    'crs': meta_10m['crs'],  # Use CRS from Sentinel-2
    'transform': meta_10m['transform'],  # Use transform from Sentinel-2
    'compress': 'lzw',  # Optional compression
    'tiled': True,      # Use tiled storage
    'BIGTIFF': 'YES'   # Enable BigTIFF
}

# Save the fused image
output_filepath = r'C:\Users\User\Documents\DataFusion\DataFusion\SimpleFusion\FusedImage\fused_image.tif'
with rasterio.open(output_filepath, 'w', **meta_fused) as dst:
    dst.write(fused_bands)

print(f'Fused image saved to {output_filepath}')


Fused image saved to C:\Users\User\Documents\DataFusion\DataFusion\SimpleFusion\FusedImage\fused_image.tif
